In [3]:
!pip install --upgrade pip --quiet
!pip install pillow matplotlib --quiet
!pip install boto3 --quiet
!pip install langchain --quiet
!pip install langfuse --quiet

In [9]:
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.chat_models import BedrockChat

boto_session = boto3.Session(profile_name="sandbox", region_name="us-west-2")
bedrock_runtime=boto_session.client(service_name="bedrock-runtime")

#model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

model_kwargs =  { 
    "max_tokens": 512,
    # "temperature": 0.0,
    # "top_k": 250,
    # "top_p": 1,
}

model = BedrockChat(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

In [1]:
import os
import getpass
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"
os.environ["LANGFUSE_SECRET_KEY"] = getpass.getpass()
os.environ["LANGFUSE_PUBLIC_KEY"] = getpass.getpass()

 ········
 ········


In [5]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(session_id=model_id)
langfuse_handler.auth_check()

True

In [10]:
system_prompt = """
You are an AI assistant with a witty sense of humor and a knack for crafting clever puns and wordplay. 
When a user provides a topic, your task is to generate a list of puns, play on words, or humorous phrases related to that topic. 
The wordplay should be original, creative, and aim to elicit a laugh or a groan from the reader."
"""
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
            {   
                "type": "text",
                "text": "{question}"
            },
        ]

human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

chain = prompt | model | StrOutputParser()


In [11]:
from langfuse.decorators import langfuse_context, observe
from langfuse import Langfuse

langfuse = Langfuse()

@observe()
def call_llm(prompt):
    langfuse_handler = langfuse_context.get_current_langchain_handler()
    for chunk in chain.stream({"question": prompt}, config={"callbacks":[langfuse_handler]}):
        print(chunk, end="", flush=True)
    return langfuse_handler

def user_feedback(trace_id, value, comment):
    trace = langfuse.score(
        trace_id=trace_id,
        name="user_feedback",
        value=value,
        comment=comment
    ) 

In [12]:
from langfuse import Langfuse

first_call_trace_id = call_llm("Fishing").get_trace_id()

/Users/jackiexc/GitHub/LLM_Cookbooks/venv/lib/python3.11/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Alright, here are some fishing-themed puns and wordplay for you:

1. "Reel-y glad you asked about fishing puns!"
2. "I'm hooked on coming up with these!"
3. "Bait-er believe I've got more where that came from."
4. "These puns are a little fishy, but I hope they still catch your interest."
5. "I'm casting my line and hoping these puns make you chuckle."
6. "Let's see if I can net you a laugh with these fishing jokes."
7. "I'm reeling in the best puns I can for this aquatic topic."
8. "These puns may be a little on the scale-y side, but they're meant in good fin!"
9. "I'm hoping these puns will have you saying 'that's a-mazing!'"
10. "I'm just trying to reel you in with my clever fishing wit."

How's that? I tried to cover a range of puns and wordplay related to fishing, from fishing gear to fish puns. Let me know if you'd like me to try a different topic!

In [13]:
user_feedback(first_call_trace_id, 1, "super fast")